## 2022 CVE CPE Review 

In [1]:
%%capture
!mkdir -p jsondata
%cd jsondata
!rm *.json 
!rm *.zip 
!wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-{2002..2022}.json.zip 
!unzip -o "*.zip" 
!wget https://raw.githubusercontent.com/CVEProject/cve-website/dev/src/assets/data/CNAsList.json

In [2]:

from IPython.core.magic import register_cell_magic
from IPython.display import Markdown
import calplot
import datetime
from datetime import date
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings

@register_cell_magic
def markdown(line, cell):
    return Markdown(cell.format(**globals()))


logging.getLogger('matplotlib.font_manager').disabled = True
warnings.filterwarnings("ignore")
pd.set_option('display.width', 500)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 10)

In [4]:
row_accumulator = []

for filename in glob.glob('nvdcve-1.1-*.json'):
    with open(filename, 'r', encoding='utf-8') as f:
        nvd_data = json.load(f)
        for cves in nvd_data['CVE_Items']:
            if cves['configurations']['nodes'] != []:
                if "children" in cves['configurations']['nodes'][0]:
                    for j in range(len(cves['configurations']['nodes'][0]['children'])):
                        for cpe in cves['configurations']['nodes'][0]['children'][j]['cpe_match']:
                            if cpe['vulnerable'] == True:
                                        cve = cves['cve']['CVE_data_meta']['ID']
                                        try:
                                            published_date = cves['publishedDate']
                                        except KeyError:
                                            published_date = 'Missing_Data'
                                        cpe_string = cpe['cpe23Uri']
                                        try:
                                            end = cpe['versionEndExcluding']
                                        except: 
                                            end = 'None'
                                        try:
                                            end2 = cpe['versionEndIncluding']
                                        except: 
                                            end2 = 'None' 
                                        try:
                                            start = cpe['versionStartExcluding']
                                        except: 
                                            start = 'None'
                                        try:
                                            start2 = cpe['versionStartIncluding']
                                        except: 
                                            start2 = 'None'     
                                        new_row = { 
                                        'CVE': cve,
                                        'Published': published_date,
                                        'CPE' : cpe_string,
                                        'StartI' : start,
                                        'StartE' : start2,
                                        'EndI' : end,
                                        'EndE' : end2 
                                        }
                                        row_accumulator.append(new_row)
        


nvd = pd.DataFrame(row_accumulator)  
nvd['Published'] = pd.to_datetime(nvd['Published'])
thisyear = ((nvd['Published'] > '2022-01-01') & (nvd['Published']  < '2022-12-31'))
nvd = nvd.loc[thisyear]
nvd = nvd.sort_values(by=['Published'])
nvd = nvd.reset_index(drop=True)
nvdcount = nvd['Published'].count()
nvdunique = nvd['Published'].nunique()
startdate = date(2022, 1, 1)
enddate  = date.today()
numberofdays = enddate - startdate 
per_day = nvdcount/numberofdays.days

### CVEs With Most CPEs


In [7]:
nvd['CVE'].value_counts().head(20)

CVE-2022-22160    364
CVE-2022-22226    240
CVE-2022-22223    222
CVE-2022-20725    215
CVE-2022-22191    208
CVE-2022-22249    197
CVE-2022-22234    174
CVE-2022-22185    154
CVE-2022-22202    150
CVE-2022-22174    147
CVE-2022-22216    146
CVE-2022-22161    143
CVE-2022-22218    136
CVE-2022-22167    133
CVE-2022-22217    132
CVE-2022-22180    128
CVE-2022-22186    123
CVE-2022-22221    115
CVE-2022-23120     83
CVE-2022-23119     83
Name: CVE, dtype: int64

## Most Common CPEs

In [8]:
nvd['CPE'].value_counts().head(20)

cpe:2.3:o:google:android:11.0:*:*:*:*:*:*:*                              299
cpe:2.3:o:google:android:12.0:*:*:*:*:*:*:*                              291
cpe:2.3:o:google:android:10.0:*:*:*:*:*:*:*                              183
cpe:2.3:a:adobe:acrobat_dc:*:*:*:*:continuous:*:*:*                      126
cpe:2.3:a:adobe:acrobat_reader_dc:*:*:*:*:continuous:*:*:*               126
cpe:2.3:o:reolink:rlc-410w_firmware:3.0.0.136_20121102:*:*:*:*:*:*:*      87
cpe:2.3:a:foxit:pdf_editor:*:*:*:*:*:*:*:*                                82
cpe:2.3:a:google:chrome:*:*:*:*:*:*:*:*                                   74
cpe:2.3:a:adobe:illustrator:*:*:*:*:*:*:*:*                               66
cpe:2.3:a:adobe:indesign:*:*:*:*:*:*:*:*                                  62
cpe:2.3:a:foxit:pdf_reader:*:*:*:*:*:*:*:*                                50
cpe:2.3:a:octopus:octopus_server:*:*:*:*:*:*:*:*                          48
cpe:2.3:o:qualcomm:ar8035_firmware:-:*:*:*:*:*:*:*                        47

In [9]:
nvd.describe()

,CVE,Published,CPE,StartI,StartE,EndI,EndE
count,10945,10945,10945,10945,10945,10945,10945
unique,4552,952,2791,5,292,719,364
top,CVE-2022-22160,2022-10-18 03:15:00+00:00,cpe:2.3:o:google:android:11.0:*:*:*:*:*:*:*,None,None,None,None
freq,364,1236,299,10941,9750,9374,9708
first,NaN,2022-01-01 05:15:00+00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,2022-12-22 20:15:00+00:00,NaN,NaN,NaN,NaN,NaN
